In [1]:
# about
created by
created on : 12 aug

SyntaxError: invalid syntax (<ipython-input-1-bdb823e7cc54>, line 2)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# All the groupby() you should know

* Pandas’ groupby() allows us to split data into separate groups to perform computations for better analysis.
* groupby process -> split-apply-combine


2. The “group by” process: split-apply-combine

3. Aggregation

4. Transformation
5. Filtration
6. Grouping by multiple categories
7. Resetting index with as_index
8. Handling missing values




## 1. What is Pandas groupby() and how to access groups information?

In [2]:
df = pd.read_csv('titanic dataset/train.csv')
df.head()

NameError: name 'pd' is not defined

* The role of groupby() is anytime we want to analyze data by some categories. 
* The simplest call must have a column name. In our example, let’s use the Sex column.

In [ ]:
df_groupby_sex = df.groupby('Sex')

* The statement literally means we would like to analyze our data by different Sex values. 
* By calling the type() function on the result, we can see that it returns a DataFrameGroupBy object

In [ ]:
type(df_groupby_sex)

* the `groupby()` function returns a dataframegroupby object
    * it essentially describes how the rows of the OG dataset have been split
* there are attributes and methods available for us to access the groups info

### Example
* we can use `ngroups` attribute to get the number of groups 


In [ ]:
df_groupby_sex.ngroups

* we can use the `groups` attribute to get the groups object .
* The integer numbers in the list correspond to row number

In [ ]:
df_groupby_sex.groups

* we can use `size()` method to compute and display group sizes

In [ ]:
df_groupby_sex.size()

* to preview groups, we can call `first()` or `last()` to preview the result with the first or last entry

In [ ]:
df_groupby_sex.first()

In [ ]:
df_groupby_sex.last()

* `get_group` : retrieve one of the created groups

In [ ]:
df_female = df_groupby_sex.get_group('female')
df_female.head()

# 2. The “group by” process: split-apply-combine

* The “group by” process involves one or more of the following steps:
    1. splitting the data into groups
    2. applying a function to each group independently
    3. combining the results into a data structure
* out of these `groupby()` is widely used for the split step and is the most straightforward

* when you get these groups, you might want to do things such as
    * <b> aggregation </b> : 
        * compute a summary statistic for each group. for example, sum, mean, or count.
    * <b> transformation </b> : 
        * perform some group-specific computations and return a like-indexed object. 
        * For example, standardize data within a group or replacing missing values within groups.
    * <b> filtration </b>:
        * discard some groups, according to a group-wise computation that evaluates True or False. 
        * For example, discard data that belongs to groups with only a few members or filter out data based on the group sum or mean.
        
        
# 3. Aggregation
* Once DataFrameGroupBy has been created, several methods are available to perform a computation on the grouped data
* Aggregation is a very common step to perform

    ## With column
    * aggregation on a specific column

In [ ]:
df.groupby('Sex').Age.max()

3. 
   ## With `agg()` method
* There is a method called agg() and it allows us to specify multiple aggregation functions at once.

In [ ]:
df.groupby('Sex').Age.agg(['max', 'min', 'count', 'median', 'mean'])

* If you would like to use a custom aggregation function:

In [ ]:
def categorize(x):
    m = x.mean()
    return True if m > 29 else False
df.groupby('Sex').Age.agg(['max', 'mean', categorize])


* We can also use a lambda expression

In [ ]:
df.groupby('Sex').Age.agg(
    ['max', 'mean', lambda x: True if x.mean() > 50 else False]
)

3. 
      ## Without column
      * you don't necessarily have to specify a column like <b> Age </b>
      * without a column, it will perform the aggregation across all of the numeric columns

In [ ]:
df.groupby('Sex').mean()

In [ ]:
df.groupby('Sex').agg(['mean', 'median'])


# 4. Transforming data

* Transformation is a process in which we perform some group-specific computations and return a like-indexed (same length) object. 
* When looking for transforming data, `transform()` and `apply()` are the most commonly used functions.
* Create a lambda expression for standardization

In [ ]:
standardization = lambda x: (x-x.mean())/ x.std()

* to perform the standardization on the Age column, we use `transform()`

In [ ]:
df.groupby('Sex').Age.transform(standardization)


* To perform the standardization on Age column using apply()

In [ ]:
df.groupby('Sex').Age.apply(standardization)

# 5. Filtration
* Let’s take a look at how to discard data that belongs to groups with only a few members.
* First, we group the data by Cabin and take a quick look at the size for each group. E.g.
    `df.groupby('Cabin').size()
    df.groupby('Cabin').filter(lambda x: len(x) >= 4)
    `

# 6. Grouping by multiple categories
*  Instead of a label, we can also pass a list of labels to work with multiple grouping.
```
# Creating a subset
df_subset = df.loc[:, ['Sex', 'Pclass', 'Age', 'Fare']]
# Group by multiple categories
df_subset.groupby(['Sex', 'Pclass']).mean()
```

## 7. Merge()

The simplest call without any key column
Specifying key columns using on
Merging using left_on and right_on
Various forms of joins: inner, left, right and outer
Using validate to avoid invalid records

### 7.1 The simplest merge.

`merge(left_dataframe, right_dataframe`

In [12]:
df_customer = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['Tom', 'Jenny', 'James', 'Dan'],
})
df_info = pd.DataFrame({
    'id': [2, 3, 4, 5],
    'age': [31, 20, 40, 70],
    'sex': ['F', 'M', 'M', 'F']
})
pd.merge(df_customer, df_info)

,id,name,age,sex
0,2,Jenny,31,F
1,3,James,20,M
2,4,Dan,40,M


* by default, the function will combine data on common columns (i.e. id)
* will produce results that matches both left and right dataframe
* below is equivalent statement if you called merge from the left dataframe
`df_customer.merge(df_info`

### 7.2 Specifying key columns using argument on
* You can specify the common columns for merging. To do so, pass an additional argument on as the name of the common column, here 'id' in our example, to merge() function:

In [13]:
pd.merge(df_customer, df_info, on='id')

,id,name,age,sex
0,2,Jenny,31,F
1,3,James,20,M
2,4,Dan,40,M


* If you use on , you have to make sure the column you specify must be present in both left and right DataFrames.
* To combine data on multiple common columns, you can pass a list toon:
`pd.merge(df_customer, df_order, on=['id', 'name'])`

### 7.3 Merging using left_on and right_on
* It might happen that the column on which you want to merge the DataFrames have different names. 
* For such merges, you will have to specify the left_on as the left DataFrame name and right_on as the right DataFrame name, for example:
```
pd.merge(
  df_customer, 
  df_info_2, 
  left_on='id', 
  right_on='customer_id'
)

```

### 7.4  Various type of joins: inner, left, right and outer

* `inner`: the default join type in Pandas merge() function and it produces records that have matching values in both DataFrames
* `left`: produces all records from the left DataFrame and the matched records from the right DataFrame
* `right`: produces all records from the right DataFrame and the matched records from the left DataFrame
* `outer`: produces all records when there is a match in either left or right DataFrame

In [17]:
df_customer = pd.DataFrame({
    'id': [1,2,3,4],
    'name': ['Tom', 'Jenny', 'James', 'Dan'],
})
df_info = pd.DataFrame({
    'id': [2,3,4,5],
    'age': [31,20,40,70],
    'sex': ['F', 'M', 'M', 'F']
})
df_customer

,id,name
0,1,Tom
1,2,Jenny
2,3,James
3,4,Dan


In [18]:
df_info

,id,age,sex
0,2,31,F
1,3,20,M
2,4,40,M
3,5,70,F


#### inner

In [19]:
pd.merge(df_customer, df_info, on='id', how='inner')

,id,name,age,sex
0,2,Jenny,31,F
1,3,James,20,M
2,4,Dan,40,M


####  left : df_customer

* The left join produces all records from the left DataFrame, and the matched records from the right DataFrame. 
* If there is no match, the left side will contain NaN. 
* You can set the argument how='left' to do left join:


In [21]:
pd.merge(df_customer, df_info, on='id', how='left')

,id,name,age,sex
0,1,Tom,NaN,NaN
1,2,Jenny,31.0,F
2,3,James,20.0,M
3,4,Dan,40.0,M


#### right: df_info
* The right join produces all records from the right DataFrame, and the matched records from the left DataFrame. 
* If there is no match, the right side will contain NaN.

In [22]:
pd.merge(df_customer, df_info, on='id', how='right')

,id,name,age,sex
0,2,Jenny,31,F
1,3,James,20,M
2,4,Dan,40,M
3,5,NaN,70,F


#### outer
* The outer join produces all records when there is a match in either left or right DataFrame.
* NaN will be filled for no match on either sides. 

In [23]:
pd.merge(df_customer, df_info, on='id', how='outer')

,id,name,age,sex
0,1,Tom,NaN,NaN
1,2,Jenny,31.0,F
2,3,James,20.0,M
3,4,Dan,40.0,M
4,5,NaN,70.0,F


### 7.5 Using validate to avoid invalid records
* The result of merge() might have an increased number of rows if the merge keys are not unique. For example

In [24]:
df_customer = pd.DataFrame({
    'id': [1,2,3,4],
    'name': ['Tom', 'Jenny', 'James', 'Dan'],
})
df_order_2 = pd.DataFrame({
    'id': [2, 2, 4, 4],
    'product': ['A', 'B' ,'A', 'C'],
    'quantity': [31, 21, 20,40],
    'date': pd.date_range('2019-02-24', periods=4, freq='D')
})

* both df_customer and df_order_2 have 4 records
* but if you merged on id you'd get 6 records ( double 2, double 4)

In [25]:
pd.merge(df_customer, df_order_2, how='left', on='id')

,id,name,product,quantity,date
0,1,Tom,NaN,NaN,NaT
1,2,Jenny,A,31.0,2019-02-24
2,2,Jenny,B,21.0,2019-02-25
3,3,James,NaN,NaN,NaT
4,4,Dan,A,20.0,2019-02-26
5,4,Dan,C,40.0,2019-02-27


#### why?
* Here are the reasons:
    * the how='left' will produce all records from df_customer, and the matched records from df_order_2.
    * In addition, the id in df_order_2 is not unique and all the matching records will be combined and returned.
* it's a case of <b>one-to-many</b> merge
* However, one-to-many might be invalid in some other cases, for example, there are two records with the id value 2 in df_info

In [26]:
df_customer = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['Tom', 'Jenny', 'James', 'Dan'],
})
df_info = pd.DataFrame({
    'id': [2, 2, 3, 4, 5],
    'age': [31, 21, 20, 40, 70],
    'sex': ['F', 'F', 'M', 'M', 'F']
})

In [27]:
pd.merge(df_customer, df_info, how='left', on='id')


,id,name,age,sex
0,1,Tom,NaN,NaN
1,2,Jenny,31.0,F
2,2,Jenny,21.0,F
3,3,James,20.0,M
4,4,Dan,40.0,M


* doing this, since the id is not unique, you get 2 different records for the same customer Jenny
* to remedy this we want a 1-1 mapping
    * we can do this by setting the argument `validate='1:1'`
        * it checks if merge keys are unique in both left/right dataframes and will raise a MergeError if the validation fails

In [28]:
pd.merge(df_customer, df_info, how='left', on='id', validate='1:1')

MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

* The argument validate takes one of the following values, so you can use it to validate different merge outputs.

    * `one_to_one` or `1:1` : check if merge keys are unique in both left and right datasets.
    * `one_to_many` or `1:m` : check if merge keys are unique in left dataset.
    * `many_to_one` or `m:1` : check if merge keys are unique in right dataset.
    * `many_to_many` or `m:m`: allowed, but does not result in checks.

----
# Apply functions using congress data

In [4]:
import pandas as pd
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df = pd.read_csv(
    "congress data/legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)

In [5]:
df.tail()

,last_name,first_name,birthday,gender,type,state,party
12044,Wright,Ron,1953-04-08,M,rep,TX,Republican
12045,Fudge,Marcia,1952-10-29,F,rep,OH,Democrat
12046,Haaland,Debra,1960-12-02,F,rep,NM,Democrat
12047,Hastings,Alcee,1936-09-05,M,rep,FL,Democrat
12048,Stivers,Steve,1965-03-24,M,rep,OH,Republican


In [6]:
df.head()

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN


In [7]:
df.info

<bound method DataFrame.info of       last_name  first_name   birthday gender type state                party
0       Bassett     Richard 1745-04-02      M  sen    DE  Anti-Administration
1         Bland  Theodorick 1742-03-21      M  rep    VA                  NaN
2         Burke     Aedanus 1743-06-16      M  rep    SC                  NaN
3       Carroll      Daniel 1730-07-22      M  rep    MD                  NaN
4        Clymer      George 1739-03-16      M  rep    PA                  NaN
...         ...         ...        ...    ...  ...   ...                  ...
12044    Wright         Ron 1953-04-08      M  rep    TX           Republican
12045     Fudge      Marcia 1952-10-29      F  rep    OH             Democrat
12046   Haaland       Debra 1960-12-02      F  rep    NM             Democrat
12047  Hastings       Alcee 1936-09-05      M  rep    FL             Democrat
12048   Stivers       Steve 1965-03-24      M  rep    OH           Republican

[12049 rows x 7 columns]>

## notes 
* You call .groupby() and pass the name of the column you want to group on, which is "state". 
* Then, you use `["last_name"]` to specify the columns on which you want to perform the actual aggregation.

In [10]:
df.groupby('state')['party'].count()

state
AK      14
AL     209
AR     115
AS       2
AZ      49
CA     367
CO      92
CT     227
DC       2
DE      94
DK       8
FL     157
GA     311
GU       4
HI      23
IA     205
ID      59
IL     485
IN     340
KS     143
KY     368
LA     196
MA     411
MD     286
ME     170
MI     292
MN     160
MO     331
MS     151
MT      53
NC     345
ND      44
NE     126
NH     176
NJ     349
NM      55
NV      55
NY    1432
OH     669
OK      93
OL       0
OR      90
PA    1035
PI       0
PR      19
RI     105
SC     244
SD      51
TN     299
TX     263
UT      55
VA     419
VI       4
VT     111
WA      96
WI     198
WV     120
WY      40
Name: party, dtype: int64

* you can more than just a single column name to `.groupby()` as the first argument. You can also use
    * a list of multiple column names
    * a dict or panda series
    * a numpy array or pandas index
    

In [11]:
# groupy by 2 columns
df.groupby(["state", "gender"])["last_name"].count()

state  gender
AK     F           0
       M          16
AL     F           4
       M         205
AR     F           5
                ... 
WI     M         198
WV     F           1
       M         119
WY     F           1
       M          39
Name: last_name, Length: 116, dtype: int64